# folium 지도상에 텍스트 표시하기

In [1]:
import numpy as np
import pandas as pd
import json, folium

In [3]:
cdf = pd.read_csv('data/서울시_범죄_결과.csv')
cdf.set_index('구별', inplace=True)
cdf.head()

,살인발생,강도발생,강간발생,절도발생,폭력발생,살인검거율,강도검거율,강간검거율,절도검거율,폭력검거율,CCTV댓수,인구수,범죄,검거
구별,,,,,,,,,,,,,,
종로구,0.25,0.43,0.26,0.33,0.35,100.00,100.00,100.00,70.56,90.87,1772,155106,1.62,100.00
중구,0.08,0.36,0.18,0.65,0.29,50.00,83.33,58.97,47.68,84.96,2333,132259,1.56,3.37
용산구,0.17,0.14,0.32,0.10,0.33,100.00,100.00,87.13,54.45,89.54,2383,240665,1.06,78.54
성동구,0.42,0.57,0.07,0.08,0.07,83.33,100.00,72.18,52.68,87.23,3602,295767,1.21,53.27
광진구,0.25,0.36,0.32,0.50,0.26,100.00,83.33,78.02,53.76,85.54,2588,355306,1.69,56.97


In [4]:
geo_str = json.load(open('data/skorea_municipalities_geo_simple.json', encoding='utf8'))

In [7]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=cdf['살인발생'],
    columns=[cdf.index, cdf.살인발생],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)

title = '<h3 align="center" style="font-size:20px">Murder</h3>'     # 서울시 구별 살인발생 비율
map.get_root().html.add_child(folium.Element(title))
map

In [14]:
from folium.features import DivIcon
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Marker(
    [37.5502, 126.982]
).add_to(map)
folium.Choropleth(
    geo_data=geo_str,
    data=cdf['살인발생'],
    columns=[cdf.index, cdf.살인발생],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
folium.map.Marker(
    [37.5502, 126.982],
    icon = DivIcon(
        icon_size=(100,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt">Seoul City</div>'
    )
).add_to(map)
map

- 지도위에 구 이름을 표시하기 위해 좌표의 평균을 구함

In [16]:
type(geo_str['features'])

list

In [36]:
gu_dict = {}
for gu in geo_str['features']:
    for coord in gu['geometry']['coordinates']:
        geo = np.array(coord)
        gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
gu_dict

{'강동구': [37.546532751824586, 127.14078218193268],
 '송파구': [37.50620055459428, 127.11197246712628],
 '강남구': [37.49679934694853, 127.0673229896189],
 '서초구': [37.47066332386016, 127.0390912771612],
 '관악구': [37.46393155080159, 126.94996289929526],
 '동작구': [37.495988209152785, 126.9530748356924],
 '영등포구': [37.51883998627326, 126.91028348590825],
 '금천구': [37.46241523438773, 126.9050257773362],
 '구로구': [37.49166908254099, 126.85870943441728],
 '강서구': [37.55703289257873, 126.83102419541086],
 '양천구': [37.529418412364095, 126.85252712500812],
 '마포구': [37.5599165285085, 126.9176866353821],
 '서대문구': [37.57597915089086, 126.94816079153975],
 '은평구': [37.61445865754635, 126.93220977883982],
 '노원구': [37.651705676071316, 127.07821750766884],
 '도봉구': [37.66711228271955, 127.03387331602003],
 '강북구': [37.646476408369075, 127.01356807286824],
 '성북구': [37.61007782092726, 127.02020916322154],
 '중랑구': [37.59847817045754, 127.09036092595524],
 '동대문구': [37.57621738848033, 127.04968775012617],
 '광진구': [37.548640

In [49]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=cdf['살인발생'],
    columns=[cdf.index, cdf.살인발생],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
for gu_name in cdf.index:
    folium.map.Marker(
        location=gu_dict[gu_name],
        icon = DivIcon(
            icon_size=(80,20),
            icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
map

- 별도의 gu_dict를 만들지 않는 방법

In [50]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=cdf['살인발생'],
    columns=[cdf.index, cdf.살인발생],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
for gu_name in cdf.index:
    for gu in geo_str['features']:
        if gu_name != gu['id']:
            continue
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            lat, lng = np.mean(geo[:,1]), np.mean(geo[:,0])
            
    folium.map.Marker(
        location=[lat, lng],
        icon = DivIcon(
            icon_size=(80,20),
            icon_anchor=(20,0),
            html=f'<div style="font-size: 10pt">{gu_name}</div>'
        )
    ).add_to(map)
map